In [1]:
import pdfplumber
import requests
import pandas as pd

**1 - Data Prepocessing** 
<br> Preliminary data cleaning and inspection before reformatting as SQL tables
<br> Involves: lowering all string, making units consistent, making data formats consistent, dropping unnecessary data, merging column data 

In [2]:
# Target data spread = 2013 - 2021, across all UK main waterways
# greenpeace labs technical report - pages 16 to 17 - PDF - Feb/March 2019 - All UK
import re

with pdfplumber.open("data/GP_mps.pdf") as pdf:
    page = pdf.pages[15]
    first_page_dat = pd.Series(page.extract_table(table_settings = {"horizontal_strategy": "text"}))
    
    sec_page = pdf.pages[16]
    second_page_dat = pd.Series(sec_page.extract_table(table_settings = {"horizontal_strategy": "text"}))
    
    page_dat = pd.concat([first_page_dat, second_page_dat])
    #im = page.to_image(resolution = 120)
    #im.show()
print(page_dat)

0     [, None, None, , None, None, , None, None, , N...
1     [, None, None, , None, None, , None, None, , T...
2     [River, None, None, Sampling location, None, N...
3     [, None, None, , None, None, , None, None, flo...
4     [, Exe, , , Footbridge over the Exe in central...
                            ...                        
27     [, , , , , , , , , , , , , , , , , , , , , , , ]
28    [, Lagan, , , , , , 28/02/2019, , , 6.38, , , ...
29    [, None, None, , below sewage outlet, , , None...
30    [, , , , , , , , , , , None, None, None, None,...
31    [, Lagan, , , Above Stranmillis Weir (NTL) in ...
Length: 95, dtype: object


In [3]:
page_dat.drop([0,1,2,3], inplace = True)

In [4]:
# Transform pdf extraction into a dataframe
# Record format: Source, Total plastic Mass (per km river flow), Microplastics (per km river flow), Sampling Dates, Sampling Location, Sampling Method
print(page_dat)
gp_dat = pd.DataFrame((i for i in page_dat), columns = ("drop 6", "samp_location", "drop1", "drop 5", "drop 18", "drop 19", "drop 20", "samp_date", "drop 7", "drop 8", "drop2", "drop 9", "drop 10", "drop3", "drop 11", "drop 12", "drop4", "drop 13", "drop 14", "sum_km2", "drop 15", "drop 16", "sum_micro_km2", "drop 17"))

4     [, Exe, , , Footbridge over the Exe in central...
5     [, None, None, , None, None, , None, None, , N...
6     [, Exe, , , Footbridge over the Exe in central...
7     [, None, None, , None, None, , None, None, , N...
8     [, Exe, , , Footbridge over the Exe in central...
                            ...                        
27     [, , , , , , , , , , , , , , , , , , , , , , , ]
28    [, Lagan, , , , , , 28/02/2019, , , 6.38, , , ...
29    [, None, None, , below sewage outlet, , , None...
30    [, , , , , , , , , , , None, None, None, None,...
31    [, Lagan, , , Above Stranmillis Weir (NTL) in ...
Length: 87, dtype: object


In [5]:
gp_dat = gp_dat.drop(columns = [col for col in gp_dat.columns if "drop" in col])
gp_dat = gp_dat.dropna()
gp_dat.reset_index(inplace = True, drop = True)

In [6]:
# Add source and sampling method details, format datetime
gp_dat["source"] = "https://www.greenpeace.to/greenpeace/wp-content/uploads/2019/06/GRL-TR-04-2019-plastics-in-UK-rivers.pdf"
gp_dat["samp_method"] = "Net Tow sampling"

gp_dat["samp_date"] = pd.to_datetime(gp_dat["samp_date"], format = '%d/%m/%Y', dayfirst = True)
gp_dat = gp_dat.reindex(columns = ["source", "samp_location", "samp_date", "sum_micro_km2", "sum_km2", "samp_method"])

for idx, i in gp_dat["samp_location"].items():
    gp_dat.loc[idx, "samp_location"] = f"River {i}"
    
gp_dat.head(n = 10)

,source,samp_location,samp_date,sum_micro_km2,sum_km2,samp_method
0,https://www.greenpeace.to/greenpeace/wp-conten...,River Exe,2019-02-06,4821,11571,Net Tow sampling
1,https://www.greenpeace.to/greenpeace/wp-conten...,River Thames,2019-02-11,52926,75938,Net Tow sampling
2,https://www.greenpeace.to/greenpeace/wp-conten...,River Thames,2019-03-12,38233,61761,Net Tow sampling
3,https://www.greenpeace.to/greenpeace/wp-conten...,River Thames,2019-03-12,14205,20518,Net Tow sampling
4,https://www.greenpeace.to/greenpeace/wp-conten...,River Thames,2019-03-12,5090,10181,Net Tow sampling
5,https://www.greenpeace.to/greenpeace/wp-conten...,River Severn,2019-02-19,12353,25940,Net Tow sampling
6,https://www.greenpeace.to/greenpeace/wp-conten...,River Severn,2019-03-04,11244,33731,Net Tow sampling
7,https://www.greenpeace.to/greenpeace/wp-conten...,River,NaT,,,Net Tow sampling
8,https://www.greenpeace.to/greenpeace/wp-conten...,River,NaT,,,Net Tow sampling
9,https://www.greenpeace.to/greenpeace/wp-conten...,River Severn,2019-03-04,0,12833,Net Tow sampling


In [7]:
# england .gov Microplastics cefas Data - CSV - 2013 to 2021 - England & Wales
cefas_dat = pd.read_csv("data/cefas_mps.csv")
cefas_dat.drop(["Rep", "Station", "Latitude", "Longitude", "Number of particles", "Number of particles Blank_Corrected", "Sediment dried weight in g","Gravel %", "Sand %", "Silt %", "OC2", "OC63", "Nit63", "Nit2"], axis = 1, inplace = True)
cefas_dat.head()

,Year,Number of particles per kg d.w. sediment,Coastal,Region
0,2013,7600,coastal,Northern North Sea
1,2013,11600,coastal,Northern North Sea
2,2013,8400,coastal,Northern North Sea
3,2013,1800,coastal,Northern North Sea
4,2013,3400,coastal,Northern North Sea


In [8]:
# Add source and sampling method details, format datetime, rename columns
cefas_dat["source"] = "https://data.cefas.co.uk/view/21717"
cefas_dat["samp_method"] = "Day Grab sediment sampler"
cefas_dat["samp_location"] = cefas_dat["Region"].astype(str) + " " + cefas_dat["Coastal"].astype(str)

In [9]:
print(cefas_dat.columns.tolist())


['Year', 'Number of particles per kg d.w. sediment ', 'Coastal', 'Region', 'source', 'samp_method', 'samp_location']


In [10]:
cefas_dat = cefas_dat.rename(columns = {"Year": "samp_date", "Number of particles per kg d.w. sediment ": "sum_micro_kg"})
cefas_dat = cefas_dat.drop(columns = ["Region", "Coastal"])
cefas_dat = cefas_dat.reindex(columns = ["source", "samp_location", "samp_date", "sum_micro_kg", "samp_method"])

cefas_dat.head()

,source,samp_location,samp_date,sum_micro_kg,samp_method
0,https://data.cefas.co.uk/view/21717,Northern North Sea coastal,2013,7600,Day Grab sediment sampler
1,https://data.cefas.co.uk/view/21717,Northern North Sea coastal,2013,11600,Day Grab sediment sampler
2,https://data.cefas.co.uk/view/21717,Northern North Sea coastal,2013,8400,Day Grab sediment sampler
3,https://data.cefas.co.uk/view/21717,Northern North Sea coastal,2013,1800,Day Grab sediment sampler
4,https://data.cefas.co.uk/view/21717,Northern North Sea coastal,2013,3400,Day Grab sediment sampler


In [15]:
# scotland .gov Microplastics in Scotlands Seas - CSV - 2013 to Present - Scotland
scots_dat = pd.read_csv("data/Scotland_mps.csv")
scots_dat.drop(["Cruise", "Season", "Field_ID", "TOW_ID", "Year", "Decimal Latitude at deployment (WGS84)", "Decimal Longitude at deployment (WGS84)", "Tow_length (m)", "WIND_SPEED", "WIND_Direction", "SEA_STATE", "Ship Speed"], axis = 1, inplace = True)
scots_dat = scots_dat.drop(columns = [col for col in scots_dat.columns if "Retrieve" in col])
scots_dat = scots_dat.drop(columns = [col for col in scots_dat.columns if "LT" in col])
scots_dat = scots_dat.drop(columns = [col for col in scots_dat.columns if "GT" in col])
scots_dat = scots_dat.drop(columns = [col for col in scots_dat.columns if "m3" in col])
scots_dat = scots_dat.drop(columns = [col for col in scots_dat.columns if "FLOW" in col])

In [16]:
scots_dat["sum_km2"] = scots_dat["SUM_micro_km2"].astype(int) + scots_dat["SUM_Macro_km2"].astype(int)
scots_dat = scots_dat.drop(columns = "SUM_Macro_km2")
scots_dat["samp_date"] = scots_dat["DATE_Deploy"].astype(str) + " " + scots_dat["TIME_Deploy"].astype(str)
scots_dat = scots_dat.drop(columns = ["DATE_Deploy", "TIME_Deploy"])
scots_dat = scots_dat.rename(columns = {"Region": "samp_location", "SUM_micro_km2": "sum_micro_km2"})
scots_dat.head()

,samp_location,sum_micro_km2,sum_km2,samp_date
0,Forth and Tay,5715,5715,06/01/2014 16:23
1,Forth and Tay,9463,9463,06/01/2014 17:24
2,Moray Firth,0,0,08/01/2014 14:01
3,Moray Firth,0,0,08/01/2014 15:40
4,Moray Firth,1022,1022,08/01/2014 18:50


In [18]:
scots_dat["source"] = "https://data.marine.gov.scot/dataset/dataset-sources-sinks-and-hazards-micro-plastics-scotland%E2%80%99s-seas-2013-present/resource#{view-graph:{graphOptions:{hooks:{processOffset:{},bindEvents:{}}}},graphOptions:{hooks:{processOffset:{},bindEvents:{}}}}"
scots_dat["samp_method"] = "Net Tow sampling"
scots_dat = scots_dat.reindex(columns = ["source", "samp_location", "samp_date", "sum_micro_km2", "sum_km2", "samp_method"])
scots_dat.head()

,source,samp_location,samp_date,sum_micro_km2,sum_km2,samp_method
0,https://data.marine.gov.scot/dataset/dataset-s...,Forth and Tay,06/01/2014 16:23,5715,5715,Net Tow sampling
1,https://data.marine.gov.scot/dataset/dataset-s...,Forth and Tay,06/01/2014 17:24,9463,9463,Net Tow sampling
2,https://data.marine.gov.scot/dataset/dataset-s...,Moray Firth,08/01/2014 14:01,0,0,Net Tow sampling
3,https://data.marine.gov.scot/dataset/dataset-s...,Moray Firth,08/01/2014 15:40,0,0,Net Tow sampling
4,https://data.marine.gov.scot/dataset/dataset-s...,Moray Firth,08/01/2014 18:50,1022,1022,Net Tow sampling


In [ ]:
# Normalise all collected data - date format, all lowercase, river/location names, spellings, units 
def normal_dat(data):
    """
    This function normalises an input DataFrame, ready for input into a larger microplastics dataset.
    
    Parameters:
    data (DataFrame): Input dataset
    
    Returns:
    norm_data (DataFrame): Normalised dataset
    """
    # lower case 
    
    pass 

**2 - Creating Database**
<br> Use SQLAlchemy for DataFrame to SQL Table
<br> Import into a SQL Database and merge into one table

**3 - Exploratory Data Analysis & Visualisation**
<br> 